### 개요
1. 문서의 내용을 읽는다
2. 문서를 쪼갠다.
    - 토근 수 초과로 답변을 생성하지 못할 수 있음
    - 문서가 길면(인풋이 길면) 답변 생성이 오래걸림
3. 임베딩 ==> 백터 데이터베이스에 저장
4. 질문이 있을 떄, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

### 참고
- Microsoft Word 공식문서
https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/

### 데이터
- 소득세법 데이터(doc) 다운로드 : https://www.law.go.kr/%EB%B2%95%EB%A0%B9/%EC%86%8C%EB%93%9D%EC%84%B8%EB%B2%95
- doc -> docx로 변환하기 : https://convertio.co/kr/doc-docx/



### docx2txt 설치

In [ ]:
# %pip install --upgrade --quiet  docx2txt langchain_community

### Text Splitter 설치

In [ ]:
# %pip install -qU langchain-text-splitters

### docx 문서 로딩하기

In [1]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1500,
    chunk_overlap=200,
)

loader = Docx2txtLoader("./tax.docx")
document = loader.load()

In [ ]:
len(document)

### 문서 split하기

In [ ]:
document_list = loader.load_and_split(text_splitter=text_splitter)

In [ ]:
document_list

### OpenAI로 임베딩하기

In [ ]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model="text-embedding-3-large")

### Chroma DB에 임베딩한 벡터 데이터 넣기

In [ ]:
# %pip install langchain-chroma

In [ ]:
from langchain_chroma import Chroma

# database = Chroma.from_documents(documents=document_list, embedding=embedding)
database = Chroma.from_documents(documents=document_list, embedding=embedding,collection_name="chroma-tax", persist_directory="./chroma")

In [ ]:
query = '연봉 5천만 원인 직장인의 소득세는 얼마인가요?'
retrived_docs = database.similarity_search(query, k=3)

In [ ]:
retrived_docs

### LLM에 질의하기
-  Augmentation을 위한 Prompt 활용

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
promt = f"""[Identity]
- 당신은 최고의 한국 소득세 전문가 입니다.
- [Context]를 참고해서 사용자의 질문에 답변해주세요.

[Context]
{retrived_docs}

Question: {query}
"""

In [ ]:
ai_message = llm.invoke(promt)

In [ ]:
ai_message.content

'연봉 5천만 원인 직장인의 소득세를 계산하려면 여러 가지 요소를 고려해야 합니다. 예를 들어, 근로소득공제, 세액공제, 인적공제 등이 있습니다. 그러나 제공된 정보에서는 구체적인 세율이나 공제 항목들이 명시되어 있지 않으므로 일반적인 연봉 5천만 원 수준에서의 소득세 계산 방식을 설명드립니다. \n\n1. **근로소득공제**: 연봉에 따라 근로소득공제를 적용합니다. 예를 들어, 5천만 원의 연봉에 대한 근로소득공제는 대략 1,100만 원 정도가 될 수 있습니다(실제 공제액은 세법에 따라 다를 수 있습니다).\n\n2. **과세표준 계산**: 근로소득에서 근로소득공제를 뺀 금액이 과세표준이 됩니다. 따라서 과세표준은 3,900만 원이 됩니다(5,000만 원 - 1,100만 원).\n\n3. **세율 적용**: 대한민국의 소득세율은 진행세(sequential tax) 구조로 되어 있습니다. 과세표준에 따라 여러 구간의 세율이 적용됩니다. 예를 들어, 2023년 기준으로 과세표준 3,900만 원에 대한 세율은 대략적으로 다음과 같습니다:\n   - 1,200만 원까지: 6%\n   - 1,200만 원 초과 4,600만 원 이하: 15%\n   구체적인 계산으로는 대략 582만 원이 됩니다.\n\n4. **세액공제 및 특별공제**: 기타 세액 공제 및 특별 활동 공제가 추가로 적용될 수 있으므로 최종 세금은 위에서 계산한 세금보다 낮아질 수 있습니다.\n\n정확한 금액을 확인하려면, 최신 세법 정보를 참조하거나 전문 세무사와 상담하는 것이 좋습니다. 위해 설명한 것은 일반적인 예시일 뿐, 특정 조건과 공제사항에 따라 다를 수 있습니다.'

### RetrievalQA(Question Answer)를 통해 LLM에 전달
- langchainhub는 prompt를 많이 준다는 이점이 있음

In [ ]:
# %pip install -U langchain langchainhub --quiet

In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt") # 프롬프트를 직접 생성하지 않고 hub에서 가져옴

In [ ]:
prompt

In [ ]:
database = Chroma(collection_name="chroma-tax", persist_directory="./chroma", embedding_function=embedding)
query = '연봉 5천만 원인 직장인의 소득세는 얼마인가요?'
llm = ChatOpenAI(model="gpt-4o")

In [ ]:
from langchain.vectorstores import Chroma

# 기존 저장된 벡터 DB 로드
vectorstore = Chroma(persist_directory="./chroma", embedding_function=embedding)

# 총 저장된 문서(임베딩)의 수 확인
print("총 문서 수:", vectorstore._collection.count())


### QA 체인 만들기

In [ ]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=database.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
ai_message = qa_chain({"query": query})

In [ ]:
ai_message